<a href="https://colab.research.google.com/github/wenjunsun/Covid-19-analysis-with-uw-ubicomp/blob/master/data_and_pre-processing/merge_mobility_with_first_case_and_shelter_in_place.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive / go to working directory
---
Mount my drive so I can read/write data from my drive. (which is good, since UW google drive has 1TB storage?)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


Go to my working directory.

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/My\ Drive/COVID\ 19\ data\ analysis

/content/drive/My Drive/COVID 19 data analysis


In [ ]:
ls

'Copy of COVID-19 US state policy database.csv'
 days_since.csv
 merge_mobility_with_first_case_and_shelter_in_place.ipynb
 socia_dist_with_shelter_date_and_first_case
 social_dist_all_trimmed.csv


# read and explore mobility data

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
data = pd.read_csv("social_dist_all_trimmed.csv")

Took about $1$ minute to read mobility data from disk, which is not bad considering its size.

In [ ]:
data.shape

(27850773, 18)

so we have $27,850,773$ data points, which is pretty large! we have $18$ features/columns, which is pretty standard

In [ ]:
data.head()

,Unnamed: 0,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home
0,0,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,80,25,5,6,NaN,752,NaN,NaN,5431.0
1,1,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,1.0,AL,Cleburne County,1029,10299598001,NaN,156,39,10,17,NaN,797,NaN,NaN,15016.0
2,2,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Cleburne County,1029,10299598001,NaN,156,39,10,17,NaN,797,NaN,NaN,15016.0
3,3,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Jefferson County,1073,10730109006,NaN,38,14,2,6,NaN,713,NaN,NaN,7419.0
4,4,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Tuscaloosa County,1125,11250103023,NaN,131,30,13,31,NaN,750,NaN,NaN,11979.0


In [ ]:
data.dtypes

Unnamed: 0                           int64
date_range_start                    object
date_range_end                      object
state                              float64
state_code                          object
cnamelong                           object
county_code                          int64
origin_census_block_group            int64
candidate_device_count             float64
device_count                         int64
completely_home_device_count         int64
part_time_work_behavior_devices      int64
full_time_work_behavior_devices      int64
delivery_behavior_devices          float64
median_home_dwell_time               int64
median_non_home_dwell_time         float64
median_percentage_time_home        float64
distance_traveled_from_home        float64
dtype: object

My interpretation of meaning of each column:

---

1.   date_range_start: the start date of a record
2.   date_range_end: the end date of a record
3.   state: the code for the state (interestingly it is a float, not an int)
4.   state_code: the text/string version of state code
5. cnamelong: county name as strings
6. county_code: unique itentifier of a county. (different states can have a county which might be named the same but they are geographically different. - Texas and Washington both have a King County)
7. origin_census_block_group: the identifier of a subgroup within each a county whose data we record at a daily basis (**about how many people are in one subgroup?**)
8. Candidate_device_count: not sure what this is? 
9. device_count: the total amount of devices that this subgroup has
10. completely_home_device_count: how many devices completely stayed at home between date_range_start and date_range_end
11. .... the rest are pretty straightforward. all are concerning certain behavior of the user.
12. distance_traveled_from_home: **is this the mean distance? what unit is it in? miles? kilometer?** 




In [ ]:
# selecting all data with county_code = 1015, and origin_census_block_group == 10150007002
# want to know whether we have data on multiple dates with a single county
data.loc[(data['county_code'] == 1015) & (data["origin_census_block_group"] == 10150007002)]

,Unnamed: 0,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home
0,0,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,80,25,5,6,NaN,752,NaN,NaN,5431.0
219247,219247,2020-01-02T00:00:00-06:00,2020-01-03T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,79,19,10,6,NaN,732,NaN,NaN,7059.0
438903,438903,2020-01-03T00:00:00-06:00,2020-01-04T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,74,17,9,10,NaN,835,NaN,NaN,6583.0
658056,658056,2020-01-04T00:00:00-06:00,2020-01-05T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,78,18,6,14,NaN,846,NaN,NaN,5732.0
877408,877408,2020-01-05T00:00:00-06:00,2020-01-06T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,81,24,8,9,NaN,585,NaN,NaN,5058.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26756187,26756187,2020-05-02T00:00:00-05:00,2020-05-03T00:00:00-05:00,1.0,AL,Calhoun County,1015,10150007002,160.0,80,24,2,1,5.0,794,63.0,90.0,7155.0
26974977,26974977,2020-05-03T00:00:00-05:00,2020-05-04T00:00:00-05:00,1.0,AL,Calhoun County,1015,10150007002,160.0,76,30,3,1,4.0,474,51.0,94.0,5869.0
27194618,27194618,2020-05-04T00:00:00-05:00,2020-05-05T00:00:00-05:00,1.0,AL,Calhoun County,1015,10150007002,160.0,85,26,8,1,6.0,736,111.0,79.0,8833.0
27414390,27414390,2020-05-05T00:00:00-05:00,2020-05-06T00:00:00-05:00,1.0,AL,Calhoun County,1015,10150007002,160.0,86,27,7,2,1.0,484,63.0,68.0,7288.0


So it turns out we have 127 records for subgroup 10150007002 in Calhoun County in AL from 1/1 - 5/6,
so we pretty much have data on every single date, which is good.

In [ ]:
data.describe()

,Unnamed: 0,state,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home
count,2.785077e+07,2.784923e+07,2.785077e+07,2.785077e+07,2.630768e+06,2.785077e+07,2.785077e+07,2.785077e+07,2.785077e+07,2.630768e+06,2.785077e+07,2.630768e+06,2.630768e+06,2.784974e+07
mean,1.392539e+07,2.857096e+01,2.865712e+04,2.865738e+11,1.913218e+02,8.204511e+01,2.378491e+01,7.387478e+00,1.206451e+01,2.128911e+00,7.046509e+02,4.298741e+01,9.067655e+01,9.924732e+03
std,8.039826e+06,1.634822e+01,1.637034e+04,1.637035e+11,2.419811e+02,9.490513e+01,2.883520e+01,1.164719e+01,2.349079e+01,2.226417e+00,2.995802e+02,7.619468e+01,1.647726e+01,6.152035e+04
min,0.000000e+00,1.000000e+00,1.001000e+03,1.001020e+10,6.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.200000e+01
25%,6.962693e+06,1.300000e+01,1.312100e+04,1.312101e+11,8.900000e+01,4.000000e+01,1.000000e+01,2.000000e+00,4.000000e+00,1.000000e+00,5.830000e+02,5.000000e+00,8.900000e+01,3.922000e+03
50%,1.392539e+07,2.800000e+01,2.816300e+04,2.816395e+11,1.350000e+02,6.200000e+01,1.800000e+01,5.000000e+00,8.000000e+00,1.000000e+00,7.470000e+02,2.200000e+01,9.700000e+01,6.023000e+03
75%,2.088808e+07,4.200000e+01,4.200300e+04,4.200348e+11,2.110000e+02,9.800000e+01,2.900000e+01,9.000000e+00,1.500000e+01,2.000000e+00,8.750000e+02,5.500000e+01,9.900000e+01,9.337000e+03
max,2.785077e+07,7.800000e+01,7.803000e+04,7.803099e+11,1.744600e+04,6.235500e+04,1.251000e+04,8.978000e+03,2.753400e+04,8.000000e+01,2.113000e+03,1.419000e+03,1.000000e+02,1.344236e+07


In [ ]:
data.isnull().sum()

Unnamed: 0                                0
date_range_start                          0
date_range_end                            0
state                                  1539
state_code                             1539
cnamelong                              1539
county_code                               0
origin_census_block_group                 0
candidate_device_count             25220005
device_count                              0
completely_home_device_count              0
part_time_work_behavior_devices           0
full_time_work_behavior_devices           0
delivery_behavior_devices          25220005
median_home_dwell_time                    0
median_non_home_dwell_time         25220005
median_percentage_time_home        25220005
distance_traveled_from_home            1033
dtype: int64

some notes:

---
1. seems like each record spans about a day, from the first 5 records I see. Don't know if this is true for other records. Maybe some of them span 2-3 days?
could check myself but not familar with date objects as of now. Maybe can return
to this later
2. we have about 80 devices on each subgroup on average, which might be trouble if we have say 10000 people in each subgroup. Then our sample size iw way too small to be able to generalize. Also our analysis will be biased toward people who can afford a phone. (some poor people don't have phones). Do hobos have phones? Are they excluded from our study?
3. median stay at home percentage time = 90%, which is pretty high!
4. there are 1539 records without state name? why did that happen? althought it is a pretty small portion - 1539/ 20 million
5. Good news is we have 0 nulls on county_code, which might help us merge data on county_code column.
6. we have a lot of nulls on median_non_home_dwell_time, which is weird, isn't it just 24 hours - median_home_dewll_time? And we have full data on median_home_dwell_time. But anyways, these two columns are highly correlated, so we just need to use one in our analysis 

# explore date_since data

In [ ]:
days_since_data = pd.read_csv("days_since.csv")

In [ ]:
days_since_data.head()

,Unnamed: 0,Date - first case,Date - first death,Date - reopening,Date - shelter in place,Date - shelter in place ends,cnamelong,county,county_code,state,state_code,state_name
0,0,2020-03-24,2020-04-07,2020-04-30,2020-04-04 00:00:00,2020-04-30,Autauga County,1.0,1001.0,1.0,AL,Alabama
1,1,2020-03-15,2020-03-29,2020-04-30,2020-04-04 00:00:00,2020-04-30,Baldwin County,3.0,1003.0,1.0,AL,Alabama
2,2,2020-04-03,2020-04-29,2020-04-30,2020-04-04 00:00:00,2020-04-30,Barbour County,5.0,1005.0,1.0,AL,Alabama
3,3,2020-03-30,2020-05-08,2020-04-30,2020-04-04 00:00:00,2020-04-30,Bibb County,7.0,1007.0,1.0,AL,Alabama
4,4,2020-03-25,2020-05-17,2020-04-30,2020-04-04 00:00:00,2020-04-30,Blount County,9.0,1009.0,1.0,AL,Alabama


In [ ]:
days_since_data.shape

(3234, 12)

In [ ]:
days_since_data.dtypes

Unnamed: 0                        int64
Date - first case                object
Date - first death               object
Date - reopening                 object
Date - shelter in place          object
Date - shelter in place ends     object
cnamelong                        object
county                          float64
county_code                     float64
state                           float64
state_code                       object
state_name                       object
dtype: object

interestingly county_code here is a float, which is different from the int type from mobility data set

In [ ]:
# want to see data on King County, (I live here!)
days_since_data[days_since_data['cnamelong'] == "King County"]

,Unnamed: 0,Date - first case,Date - first death,Date - reopening,Date - shelter in place,Date - shelter in place ends,cnamelong,county,county_code,state,state_code,state_name
2657,2657,NaN,NaN,2020-05-01,2020-04-02 00:00:00,NaN,King County,269.0,48269.0,48.0,TX,Texas
2970,2970,2020-01-22,2020-02-29,NaN,2020-03-23 00:00:00,NaN,King County,33.0,53033.0,53.0,WA,Washington


shelter in place date pretty accurage on my county at least - I remember vivdly in March when whole Washignton state is order to stay at home, which coincides with 3-23-2020 in our dataset.

In [ ]:
# want to find out how many distinct county code are there
len(set(list(days_since_data['county_code'])))

3234

- So there are 3234 distinct counties, and we have 3234 data points, so each data point corresponds to exactly 1 county, which is good.
- there are 3141 counties in America in 2014, so I think our data pretty much covers every county.

In [ ]:
days_since_data.isnull().sum()

Unnamed: 0                         0
Date - first case                194
Date - first death              1419
Date - reopening                 464
Date - shelter in place          518
Date - shelter in place ends    1746
cnamelong                          0
county                             0
county_code                        0
state                              0
state_code                         0
state_name                         0
dtype: int64

looks like we have about 5% of missing data on date of first case, and 15% missing date on date of shelter in place (some states never implemented stay at home orders.). We have about 50% missing data on first death and when shelter in place ends. (which makes sense since some counties are not reopenning yet)

# Merge first case date and shelter in place date into mobility data set

In [ ]:
# these are the columns we want to insert to mobility data set
days_since_data = days_since_data[['Date - first case', 'Date - shelter in place', 'county_code']]

In [ ]:
# need to convert type of county_code to int to match the type of county_code
# in mobility data set
days_since_data.county_code = days_since_data.county_code.astype(int)

In [ ]:
days_since_data

,Date - first case,Date - shelter in place,county_code
0,2020-03-24,2020-04-04 00:00:00,1001
1,2020-03-15,2020-04-04 00:00:00,1003
2,2020-04-03,2020-04-04 00:00:00,1005
3,2020-03-30,2020-04-04 00:00:00,1007
4,2020-03-25,2020-04-04 00:00:00,1009
...,...,...,...
3229,2020-03-16,2020-03-15 00:00:00,72151
3230,2020-03-16,2020-03-15 00:00:00,72153
3231,2020-03-16,NaN,78010
3232,2020-03-16,NaN,78020


In [ ]:
data = pd.merge(data, days_since_data, how ="left", on="county_code")

In [ ]:
data.head()

,Unnamed: 0,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,0,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,80,25,5,6,NaN,752,NaN,NaN,5431.0,2020-03-18,2020-04-04 00:00:00
1,1,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,1.0,AL,Cleburne County,1029,10299598001,NaN,156,39,10,17,NaN,797,NaN,NaN,15016.0,2020-03-25,2020-04-04 00:00:00
2,2,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Cleburne County,1029,10299598001,NaN,156,39,10,17,NaN,797,NaN,NaN,15016.0,2020-03-25,2020-04-04 00:00:00
3,3,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Jefferson County,1073,10730109006,NaN,38,14,2,6,NaN,713,NaN,NaN,7419.0,2020-03-13,2020-04-04 00:00:00
4,4,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Tuscaloosa County,1125,11250103023,NaN,131,30,13,31,NaN,750,NaN,NaN,11979.0,2020-03-14,2020-04-04 00:00:00


In [ ]:
# now we have same amount of rows but 20 columns instead of 18
data.shape

(27850773, 20)

In [ ]:
# sanity check if merge really happened
days_since_data[days_since_data['county_code'].isin([1015, 1029, 1073, 1125])]

,Date - first case,Date - shelter in place,county_code
7,2020-03-18,2020-04-04 00:00:00,1015
14,2020-03-25,2020-04-04 00:00:00,1029
36,2020-03-13,2020-04-04 00:00:00,1073
62,2020-03-14,2020-04-04 00:00:00,1125


these dates do match the ones appearing on the first 5 rows on our new data dataframe, so I think our merge is successful.

In [ ]:
data.isna().sum()

Unnamed: 0                                0
date_range_start                          0
date_range_end                            0
state                                  1539
state_code                             1539
cnamelong                              1539
county_code                               0
origin_census_block_group                 0
candidate_device_count             25220005
device_count                              0
completely_home_device_count              0
part_time_work_behavior_devices           0
full_time_work_behavior_devices           0
delivery_behavior_devices          25220005
median_home_dwell_time                    0
median_non_home_dwell_time         25220005
median_percentage_time_home        25220005
distance_traveled_from_home            1033
Date - first case                    786199
Date - shelter in place             1361452
dtype: int64

turns out we have a lot of missing values of first case date and shelter in place date when we merge. This seems like a good point to save our new merged data

In [ ]:
data.to_csv("social_dist_with_dates")

check null values on first case date

In [ ]:
data[data.isnull()['Date - first case']].head()

,Unnamed: 0,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
131,131,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,29.0,MO,Dade County,29057,290574802002,NaN,28,10,2,3,NaN,687,NaN,NaN,12837.0,NaN,2020-04-06 00:00:00
147,147,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Bronx County,36005,360050237043,NaN,46,21,3,1,NaN,688,NaN,NaN,2846.0,NaN,2020-03-22 00:00:00
148,148,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Bronx County,36005,360050449012,NaN,31,11,1,1,NaN,731,NaN,NaN,3787.0,NaN,2020-03-22 00:00:00
151,151,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Kings County,36047,360470135001,NaN,17,7,2,2,NaN,1238,NaN,NaN,13308.0,NaN,2020-03-22 00:00:00
152,152,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Kings County,36047,360470404001,NaN,60,18,3,10,NaN,862,NaN,NaN,1683.0,NaN,2020-03-22 00:00:00


In [ ]:
days_since_data[days_since_data['county_code'].isin([29057,36005,36047])]

,Date - first case,Date - shelter in place,county_code
1511,NaN,2020-04-06 00:00:00,29057
1830,NaN,2020-03-22 00:00:00,36005
1851,NaN,2020-03-22 00:00:00,36047


So in original days_since_data the value is Null, so afeter merging they are null as well, which is what we expect.

# create days_since_first_case column in place of daye of first case by doing some arithematic
- we are going to do calculation with date_range_start, not date_range_end.
- for example if a record starts from 4-1-2020, and its first case date is 3-1, our new column will be 30 (days since first case)
- if the record starts from 1-1-2020, and first case is 3-1, we will have None/NaN on this new column 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cd drive/My\ Drive/COVID\ 19\ data\ analysis/

/content/drive/My Drive/COVID 19 data analysis


In [ ]:
# reread data from disk.
data = pd.read_csv("social_dist_with_dates")

In [ ]:
data.head()

,Unnamed: 0,Unnamed: 0.1,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,0,0,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Calhoun County,1015,10150007002,NaN,80,25,5,6,NaN,752,NaN,NaN,5431.0,2020-03-18,2020-04-04 00:00:00
1,1,1,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,1.0,AL,Cleburne County,1029,10299598001,NaN,156,39,10,17,NaN,797,NaN,NaN,15016.0,2020-03-25,2020-04-04 00:00:00
2,2,2,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Cleburne County,1029,10299598001,NaN,156,39,10,17,NaN,797,NaN,NaN,15016.0,2020-03-25,2020-04-04 00:00:00
3,3,3,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Jefferson County,1073,10730109006,NaN,38,14,2,6,NaN,713,NaN,NaN,7419.0,2020-03-13,2020-04-04 00:00:00
4,4,4,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,1.0,AL,Tuscaloosa County,1125,11250103023,NaN,131,30,13,31,NaN,750,NaN,NaN,11979.0,2020-03-14,2020-04-04 00:00:00


In [ ]:
data.loc[0, 'date_range_start']

'2020-01-01T00:00:00-06:00'

In [ ]:
type(data.loc[0, 'date_range_start'])

str

In [ ]:
type(data.loc[0, 'date_range_end'])

str

In [ ]:
type(data.loc[0, 'Date - first case'])

str

Oh God the dates are all strings, seems like I need to do some conversion from string to dates and ultimately have a TimeDelta object on a new column. Sigh!

In [ ]:
from datetime import datetime

In [ ]:
date = datetime.strptime(data.loc[1,'date_range_end'][:19], "%Y-%m-%dT%H:%M:%S")

In [ ]:
date

datetime.datetime(2020, 1, 2, 0, 0)

In [ ]:
# a function that given a row return the days_since_first_case.
def get_days_since_first_case(row):
  record_date_string = row['date_range_start']
  first_case_date_string = row['Date - first case']

  # if any one date is null, just return None, (or we can return special NaN value as well)
  if pd.isnull(record_date_string) or pd.isnull(first_case_date_string):
    return None

  record_date = datetime.strptime(record_date_string[:19], "%Y-%m-%dT%H:%M:%S")
  first_case_date = datetime.strptime(first_case_date_string, "%Y-%m-%d")
  time_delta = record_date - first_case_date

  # will return an int in this case
  return time_delta.days

In [ ]:
data.iloc[0]

Unnamed: 0                                                 0
Unnamed: 0.1                                               0
date_range_start                   2020-01-01T00:00:00-06:00
date_range_end                     2020-01-02T00:00:00-06:00
state                                                      1
state_code                                                AL
cnamelong                                     Calhoun County
county_code                                             1015
origin_census_block_group                        10150007002
candidate_device_count                                   NaN
device_count                                              80
completely_home_device_count                              25
part_time_work_behavior_devices                            5
full_time_work_behavior_devices                            6
delivery_behavior_devices                                NaN
median_home_dwell_time                                   752
median_non_home_dwell_ti

In [ ]:
get_days_since_first_case(data.iloc[0])

-77

Hope this format is okay, since first record has date_range_start of 1/1,
and its first case is 3/18, we have -77 as the date differnece between
this date and first case date. We could also just return None in this case..

In [ ]:
data.iloc[15990000]

Unnamed: 0                                          15990000
Unnamed: 0.1                                        15990000
date_range_start                   2020-03-13T00:00:00-04:00
date_range_end                     2020-03-14T00:00:00-04:00
state                                                     10
state_code                                                DE
cnamelong                                  New Castle County
county_code                                            10003
origin_census_block_group                       100030163021
candidate_device_count                                   NaN
device_count                                             160
completely_home_device_count                              43
part_time_work_behavior_devices                           18
full_time_work_behavior_devices                           16
delivery_behavior_devices                                NaN
median_home_dwell_time                                   799
median_non_home_dwell_ti

In [ ]:
get_days_since_first_case(data.iloc[15990000])

2

our custom function is correct in this case since the start date is 3/13,
and first case is 3/11. There is a 2 day differnece.

In [ ]:
# look at data where first case date is null.
data[data['Date - first case'].isnull()][:5]

,Unnamed: 0,Unnamed: 0.1,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
131,131,131,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,29.0,MO,Dade County,29057,290574802002,NaN,28,10,2,3,NaN,687,NaN,NaN,12837.0,NaN,2020-04-06 00:00:00
147,147,147,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Bronx County,36005,360050237043,NaN,46,21,3,1,NaN,688,NaN,NaN,2846.0,NaN,2020-03-22 00:00:00
148,148,148,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Bronx County,36005,360050449012,NaN,31,11,1,1,NaN,731,NaN,NaN,3787.0,NaN,2020-03-22 00:00:00
151,151,151,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Kings County,36047,360470135001,NaN,17,7,2,2,NaN,1238,NaN,NaN,13308.0,NaN,2020-03-22 00:00:00
152,152,152,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,36.0,NY,Kings County,36047,360470404001,NaN,60,18,3,10,NaN,862,NaN,NaN,1683.0,NaN,2020-03-22 00:00:00


In [ ]:
type(get_days_since_first_case(data.iloc[131]))

NoneType

Seems like our custom function handles None type perfectly too. So we can proceed to use this function on every row.

In [ ]:
# the following apply function consumes a huge amount of memory.. > 13 GB, 
# need to think of another way.
# data['days_since_first_case'] = data.apply(get_days_since_first_case, axis=1)

Maybe delete columns that are redundant now to save memory For now,
and when we have the resulting column, append that back to main data?

In [ ]:
small_data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'date_range_end','state','state_code','cnamelong','county_code',
                                'origin_census_block_group', 'candidate_device_count', 'device_count', 'completely_home_device_count',
                                'part_time_work_behavior_devices', 'full_time_work_behavior_devices', 'delivery_behavior_devices',
                                'median_home_dwell_time', 'median_non_home_dwell_time', 'median_percentage_time_home', 
                                'distance_traveled_from_home'])

In [ ]:
small_data.to_csv("small_data.csv")

In [ ]:
data['days_since_first_case'] = small_data.apply(get_days_since_first_case, axis = 1)

In [ ]:
data.iloc[0]

Unnamed: 0                                                 0
Unnamed: 0.1                                               0
date_range_start                   2020-01-01T00:00:00-06:00
date_range_end                     2020-01-02T00:00:00-06:00
state                                                      1
state_code                                                AL
cnamelong                                     Calhoun County
county_code                                             1015
origin_census_block_group                        10150007002
candidate_device_count                                   NaN
device_count                                              80
completely_home_device_count                              25
part_time_work_behavior_devices                            5
full_time_work_behavior_devices                            6
delivery_behavior_devices                                NaN
median_home_dwell_time                                   752
median_non_home_dwell_ti

In [ ]:
# the function that will return the days since shelter is in place.
def get_days_since_shelter(row):
  record_date_string = row['date_range_start']
  shelter_date_string = row['Date - shelter in place']

  # if any one date is null, just return None, (or we can return special NaN value as well)
  if pd.isnull(record_date_string) or pd.isnull(shelter_date_string):
    return None

  record_date = datetime.strptime(record_date_string[:19], "%Y-%m-%dT%H:%M:%S")
  shelter_date = datetime.strptime(shelter_date_string[:10], "%Y-%m-%d")
  time_delta = record_date - shelter_date

  # will return an int in this case
  return time_delta.days

In [ ]:
data['days_since_shelter'] = small_data.apply(get_days_since_shelter, axis = 1)

In [ ]:
data.iloc[0]

Unnamed: 0                                                 0
Unnamed: 0.1                                               0
date_range_start                   2020-01-01T00:00:00-06:00
date_range_end                     2020-01-02T00:00:00-06:00
state                                                      1
state_code                                                AL
cnamelong                                     Calhoun County
county_code                                             1015
origin_census_block_group                        10150007002
candidate_device_count                                   NaN
device_count                                              80
completely_home_device_count                              25
part_time_work_behavior_devices                            5
full_time_work_behavior_devices                            6
delivery_behavior_devices                                NaN
median_home_dwell_time                                   752
median_non_home_dwell_ti

In [ ]:
data.iloc[15990000]

Unnamed: 0                                          15990000
Unnamed: 0.1                                        15990000
date_range_start                   2020-03-13T00:00:00-04:00
date_range_end                     2020-03-14T00:00:00-04:00
state                                                     10
state_code                                                DE
cnamelong                                  New Castle County
county_code                                            10003
origin_census_block_group                       100030163021
candidate_device_count                                   NaN
device_count                                             160
completely_home_device_count                              43
part_time_work_behavior_devices                           18
full_time_work_behavior_devices                           16
delivery_behavior_devices                                NaN
median_home_dwell_time                                   799
median_non_home_dwell_ti

In [ ]:
data.iloc[131]

Unnamed: 0                                               131
Unnamed: 0.1                                             131
date_range_start                   2020-01-01T00:00:00-06:00
date_range_end                     2020-01-02T00:00:00-06:00
state                                                     29
state_code                                                MO
cnamelong                                        Dade County
county_code                                            29057
origin_census_block_group                       290574802002
candidate_device_count                                   NaN
device_count                                              28
completely_home_device_count                              10
part_time_work_behavior_devices                            2
full_time_work_behavior_devices                            3
delivery_behavior_devices                                NaN
median_home_dwell_time                                   687
median_non_home_dwell_ti

seems like the 2 columns we add look right on some data points I selected! Great.

In [ ]:
data.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)

In [ ]:
data.to_csv("social_dist_with_days_since")

In [ ]:
data.dtypes

date_range_start                    object
date_range_end                      object
state                              float64
state_code                          object
cnamelong                           object
county_code                          int64
origin_census_block_group            int64
candidate_device_count             float64
device_count                         int64
completely_home_device_count         int64
part_time_work_behavior_devices      int64
full_time_work_behavior_devices      int64
delivery_behavior_devices          float64
median_home_dwell_time               int64
median_non_home_dwell_time         float64
median_percentage_time_home        float64
distance_traveled_from_home        float64
Date - first case                   object
Date - shelter in place             object
days_since_first_case              float64
days_since_shelter                 float64
dtype: object

turns out for whatever reason the days_since_first_case and days_since_shelter are floats. If want them to be ints can convert them later.

From CJ: need to **groupby** county_code to **sum** up all mobility data for a single county, since our analysis is on each county.
- Next step: we want to group all the county data on the same date together. - we want to group data of the same date_range_start, date_range_end, and county_code 